<a href="https://colab.research.google.com/github/hmghaly/ai/blob/main/ai_un.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/AI_UN' #directory where we keep the data
if not os.path.exists(cwd): os.makedirs(cwd)  #if the directory doesn't exist, create it
os.chdir(cwd) #change current work directory to the directory where we keep the data
!pwd

Mounted at /content/drive
/content/drive/MyDrive/AI_UN


In [ ]:
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils

Cloning into 'code_utils'...
remote: Enumerating objects: 2178, done.
remote: Counting objects: 100% (338/338), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 2178 (delta 210), reused 304 (delta 191), pack-reused 1840
Receiving objects: 100% (2178/2178), 29.26 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (1052/1052), done.


In [ ]:
!rm -r GAresolutions
!git clone https://github.com/UNxml/GAresolutions.git GAresolutions

rm: cannot remove 'GAresolutions': No such file or directory
Cloning into 'GAresolutions'...
remote: Enumerating objects: 9403, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 9403 (delta 210), reused 246 (delta 183), pack-reused 9108
Receiving objects: 100% (9403/9403), 61.46 MiB | 13.33 MiB/s, done.
Resolving deltas: 100% (7487/7487), done.
Updating files: 100% (5484/5484), done.


#XML resolutions

In [ ]:
from code_utils.web_lib import *
from code_utils.general import *

#data_keys=["docnumber","doctitle","session","docdate","inline","docauthority","keyword","step","analysis","parliamentary"]
data_keys=["docnumber","doctitle","session","docdate","inline","docauthority",
           "keyword","step","voting","count","workflow","classification","docketnumber",
           "docstatus","docproponent","preface","formula","container","authorialnote",
           "paragraph","subheading","preamble"]
def extract_data_items(xml_path):
  data_list0=[]
  fopen=open(xml_path)
  xml_content=fopen.read()
  fopen.close()
  dom_obj=DOM(xml_content)
  items=list(dom_obj.tag_dict.items())
  for key0,val0 in items:
    if key0=="": continue
    if key0.startswith("text_node"): continue
    if key0.startswith("akomantoso"): continue
    if key0.startswith('!--'): continue
    tag_name=key0.rstrip("0123456789_")
    attrs0=val0.attrs
    inner0=val0.inner_html
    if tag_name in ["voting","preamble"]: inner0=[""]
    elif tag_name=="count":
      count_dom_obj=DOM(inner0)
      count_tag_dict_items=list(count_dom_obj.tag_dict.items())
      count_tag_dict_items.sort()
      count_tag_dict_items=[b.inner_html for a,b in count_tag_dict_items if a.startswith("organization")]
      inner0=count_tag_dict_items
    else:
      inner0=[remove_tags(inner0).strip()] 

    
    if tag_name in data_keys:
      # print(tag_name)
      # print(attrs0)
      # print(inner0)
      # print("===========")
      data_list0.append((tag_name,attrs0,inner0))
  return data_list0
    #print(key0,val0.attrs)
    #print([val0.inner_html[:200]])
def dict2html(dict0):
  html0=""
  for a,b in dict0.items():
    line0='<b>%s</b>: %s<br>'%(a,b)
    html0+=line0
  return html0

#xml_filename='GAresolutions/77session/English/A_RES_77_100-2228244E.xml'
#xml_filename='GAresolutions/77session/English/A_RES_77_104-2228250E.xml'
xml_filename='GAresolutions/77session/English/A_RES_77_149-2228534E.xml'

data_list1=extract_data_items(xml_filename)
table_content0=""
for i0,tag_item0 in enumerate(data_list1):
  tag_name0,tag_attrs0,tag_content0=tag_item0
  table_class="table-light"
  if i0%2!=0: table_class="table-dark text-dark"
  
  if tag_name0 in ['voting',"doctitle","workflow","classification"]: table_class="table-primary"
  if tag_name0 in ["preamble","authorialnote","docdate"]: table_class="table-info"
  if tag_attrs0.get("refersTo")=='inFavour': table_class="table-success"
  if tag_attrs0.get("refersTo")=='against': table_class="table-danger"
  if tag_attrs0.get("refersTo")=='abstaining': table_class="table-warning"
  attrs_html0=dict2html(tag_attrs0)
  content_html="; ".join(tag_content0)
  cur_tr0='<tr class="%s"><td>%s</td><td>%s</td><td>%s</td></tr>'%(table_class,tag_name0,attrs_html0,content_html)
  table_content0+=cur_tr0



template_fpath="machine_readable_table_template.html"
template_fopen=open(template_fpath)
template_content=template_fopen.read()
template_fopen.close()
template_dom_obj=DOM(template_content)
repl_dict={"#data_display_table_body":table_content0}
out_html=template_dom_obj.replace(repl_dict)

out_fopen=open("mrdoc-initial.html","w")
out_fopen.write(out_html)
out_fopen.close()


# fopen=open(xml_filename)
# xml_content=fopen.read()
# fopen.close()
# dom_obj=DOM(xml_content)

# items=list(dom_obj.tag_dict.items())
# for a,b in items:
#   if a=="": continue
#   if a.startswith("text_node"): continue
#   if a.startswith("akomantoso"): continue
#   if a.startswith('!--'): continue
#   print(a,b.attrs)
#   print([b.inner_html[:200]])

#docnumber_0, doctitle_0,  
#session_0 {'refersTo': '#generalAssembly', 'value': '77', 'eId': 'coverPage_1__session_1'}
#docdate_0 {'date': '2022-12-19', 'class': 'rightSmall', 'refersTo': '#dateOfIssuance'}
#inline_0 {'title': '77', 'name': 'agenda item'}
#docauthority_2 {'refersTo': '#issuingBody', 'class': 'rightBig'} ['General Assembly']
#docauthority_0 {'refersTo': '#publisher'} ['United Nations']
#keyword_74 {'refersTo': 'agenda', 'value': '77', 'showAs': 'Report of the International Law Commission on the work of its 73rd session.', 'shortForm': 'INTERNATIONAL LAW', 'dictionary': 'A/77/251'}
#keyword_68 {'value': 'ARMED CONFLICTS', 'showAs': 'ARMED CONFLICTS', 'dictionary': 'UNBIS Thesaurus'}
#keyword_62 {'value': 'full_employment', 'href': 'http://metadata.un.org/thesaurus/1002563', 'showAs': 'full_employment', 'dictionary': 'UNBIS'}
#classification_4 {'source': 'http://metadata.un.org/sdg'}
#workflow_0 {'source': '#un'}
#step_0 {'date': '2022-10-27', 'outcome': 'draftResolution', 'as': 'A/C.6/77/L.8', 'by': 'the representative of Thailand, on behalf of the Bureau'}
#step_1 {'date': '0001-01-01', 'outcome': 'draftResolutionAdditionalSponsors'}
#step_2 {'date': '0001-01-01', 'outcome': 'recommendedForAdoption', 'as': 'A/77/413', 'by': '" refersTo='}
#step_3 {'date': '2022-12-07', 'outcome': 'adopted', 'as': 'A/77/PV.47'}
#analysis_0 {'source': '#un'}
#parliamentary_0 {}
#voting_0 {'outcome': 'adoptedWithoutVote'} voting_0 {'outcome': 'adoptedRecordedVote'}
#count_0 {'value': '124', 'refersTo': 'inFavour'}
#count_2 {'value': '3', 'refersTo': 'against'}
#count_4 {'value': '42', 'refersTo': 'abstaining'}






##Now processing all resolutions

In [ ]:
import json
from code_utils.general import *
cur_dir='GAresolutions/77session/English'
output_dir="output/res_json"
if not os.path.exists(output_dir): os.makedirs(output_dir)
files0=get_dir_files(cur_dir,"xml")

for fpath0 in files0:
  fname0=os.path.split(fpath0)[-1]
  json_fname=fname0.replace(".xml",".json")
  json_fpath=os.path.join(output_dir,json_fname)
  if os.path.exists(json_fpath): continue

  print(fpath0,fname0)
  data_list0=extract_data_items(fpath0)
  out_fopen=open(json_fpath,"w")
  data_json=json.dump(data_list0,out_fopen)
  out_fopen.close()

#extract_data_items
# xml_filename='GAresolutions/77session/English/A_RES_77_149-2228534E.xml'
# files=""

GAresolutions/77session/English/A_RES_77_19-2226541E.xml A_RES_77_19-2226541E.xml
GAresolutions/77session/English/A_RES_77_190-2228886E.xml A_RES_77_190-2228886E.xml
GAresolutions/77session/English/A_RES_77_191-2228887E.xml A_RES_77_191-2228887E.xml
GAresolutions/77session/English/A_RES_77_192-2228888E.xml A_RES_77_192-2228888E.xml
GAresolutions/77session/English/A_RES_77_193-2228890E.xml A_RES_77_193-2228890E.xml
GAresolutions/77session/English/A_RES_77_194-2228891E.xml A_RES_77_194-2228891E.xml
GAresolutions/77session/English/A_RES_77_195-2228892E.xml A_RES_77_195-2228892E.xml
GAresolutions/77session/English/A_RES_77_196-2228893E.xml A_RES_77_196-2228893E.xml
GAresolutions/77session/English/A_RES_77_197-2228894E.xml A_RES_77_197-2228894E.xml
GAresolutions/77session/English/A_RES_77_198-2228911E.xml A_RES_77_198-2228911E.xml
GAresolutions/77session/English/A_RES_77_199-2228912E.xml A_RES_77_199-2228912E.xml
GAresolutions/77session/English/A_RES_77_2-2223393E.xml A_RES_77_2-2223393E.xm

##Indexing data for resolutions

In [ ]:
import json
from itertools import groupby
from code_utils.general import *
output_dir="output/res_json"
json_files=get_dir_files(output_dir)
#cur_json_fpath=json_files[0]
main_data_dict={}
fwd_index=[]
for cur_fpath in json_files:
  print(cur_fpath)
  json_fopen=open(cur_fpath)
  json_data=json.load(json_fopen)
  json_fopen.close()


  keywords=[]
  sdgs=[]
  voted_for=[]
  voted_against=[]
  abstained=[]
  doc_data_dict={}
  for tag_name0,tag_attr0,tag_content0 in json_data:
    if tag_name0=="keyword": 
      #print(tag_name0,tag_attr0,tag_content0)
      key0=tag_name0
      val0=tag_attr0.get("showAs")
      if val0!=None: 
        #print(key0,val0,tag_attr0)
        if val0.startswith("sdg_"): sdgs.append(val0)
        else: keywords.append(val0)
        
      #print(key0,val0)
    elif tag_name0=="doctitle":
      refers_to=tag_attr0.get('refersTo')
      if refers_to==None:
        key0=tag_name0
        val0=tag_content0[0]
      else:
        key0=refers_to.strip("#")
        val0=tag_content0[0]
      doc_data_dict[key0]=val0
    elif tag_name0=="docnumber":
      refers_to=tag_attr0.get('refersTo')
      if refers_to!=None:
        key0=tag_name0
        val0=tag_content0[0]
        doc_data_dict[key0]=val0
    elif tag_name0=="count":
      #print(tag_name0,tag_attr0)
      refers_to=tag_attr0.get('refersTo')
      if refers_to=='inFavour': voted_for.extend(tag_content0)
      if refers_to=='against': voted_against.extend(tag_content0)
      if refers_to=='abstaining': abstained.extend(tag_content0)

    else: 
      pass
      #print(tag_name0,tag_attr0,tag_content0)

  keywords=sorted(list(set(keywords)))
  doc_data_dict["keywords"]=keywords
  doc_data_dict["sdgs"]=sdgs
  doc_data_dict["voted_for"]=voted_for
  doc_data_dict["voted_against"]=voted_against
  doc_data_dict["abstained"]=abstained      
  for kw in keywords:
    fwd_index.append(("keyword", kw,cur_fpath))
  for item0 in sdgs:
    fwd_index.append(("sdg", item0,cur_fpath))
  for item0 in voted_for:
    fwd_index.append(("voted_for", item0,cur_fpath))
  for item0 in voted_against:
    fwd_index.append(("voted_against", item0,cur_fpath))
  for item0 in abstained:
    fwd_index.append(("abstained", item0,cur_fpath))

  main_data_dict[cur_fpath]=doc_data_dict

  # print("keywords:", keywords)
  # print("sdgs:", sdgs)
  # print("voted_for:", voted_for)
  # print("voted_against:", voted_against)
  # print("abstained:", abstained)
  # # for a,b in doc_data_dict.items():
  # #   print(a,b)
  # # print("------")

  # print("=========")

fwd_index.sort()
final_inv_index_dict={}
grouped_by_main=[(key,[v[1:] for v in list(group)]) for key,group in groupby(fwd_index,lambda x:x[0])]
for key0,grp0 in grouped_by_main:
  grp0.sort()
  grouped_by_seccondary=[(key,[v[1] for v in list(group)]) for key,group in groupby(grp0,lambda x:x[0])]
  grouped_by_seccondary_dict=dict(iter(grouped_by_seccondary))
  final_inv_index_dict[key0]=grouped_by_seccondary_dict
  # print(key0, len(grp0))
  # for key1,grp1 in grouped_by_seccondary[:10]:
  #   print(key1,len(grp1),grp1[:5])
#inv_index=dict(iter(grouped))
# for key1,grp1 in grouped:
#   print(key1,grp1)
print("json_files",len(json_files))

output/res_json/A_RES_77_1-2223391E.json
output/res_json/A_RES_77_10-2225538E.json
output/res_json/A_RES_77_100-2228244E.json
output/res_json/A_RES_77_101-2228245E.json
output/res_json/A_RES_77_102-2228247E.json
output/res_json/A_RES_77_103-2228249E.json
output/res_json/A_RES_77_104-2228250E.json
output/res_json/A_RES_77_105-2228251E.json
output/res_json/A_RES_77_106-2228252E.json
output/res_json/A_RES_77_107-2228253E.json
output/res_json/A_RES_77_108-2228254E.json
output/res_json/A_RES_77_109-2228255E.json
output/res_json/A_RES_77_11-2226533E.json
output/res_json/A_RES_77_110-2228256E.json
output/res_json/A_RES_77_111-2228268E.json
output/res_json/A_RES_77_112-2228269E.json
output/res_json/A_RES_77_113-2228275E.json
output/res_json/A_RES_77_114-2228280E.json
output/res_json/A_RES_77_115-2228288E.json
output/res_json/A_RES_77_116-2228309E.json
output/res_json/A_RES_77_117-2228325E.json
output/res_json/A_RES_77_118-2228394E.json
output/res_json/A_RES_77_119-2228481E.json
output/res_json

In [ ]:
def AND(vals):
  vals=[v for v in vals if v!=None]
  if vals==[]: return []
  cur_results=vals[0]
  for v1 in vals[1:]:
    cur_results=list(set(cur_results).intersection(set(v1)))
  return cur_results

for i0 in range(1,18):
  sdg_str="sdg_%s"% i0   
  print(sdg_str)
  query_dict={}
  query_dict["sdg"]=sdg_str #'sdg_2'
  query_dict["keyword"]=None
  query_dict["voted_for"]=None #"CANADA"
  query_dict["voted_against"]=None #"CANADA" #'EGYPT' #'AFGHANISTAN' #'ISRAEL' "URUGUAY" #
  query_dict["abstained"]="CANADA"
  output_res_dict={}
  for query_key,query_val in query_dict.items():
    output_res_dict[query_key]=final_inv_index_dict[query_key].get(query_val,[])
  cur_vals=[]
  for query_key,query_val in query_dict.items():
    if query_val==None: continue
    query_res=output_res_dict.get(query_key,[])
    #print(query_key,query_val,len(query_res),query_res[:10])
    cur_vals.append(query_res)

  combined=AND(cur_vals)
  print("Number of results",len(combined))
  print("================")
  for res_path0 in combined:
    #print(res_path0)
    cur_data0=main_data_dict[res_path0]
    resolutionTitle=cur_data0.get("resolutionTitle")
    docnumber=cur_data0.get("docnumber")
    print(docnumber, resolutionTitle)
    #print(resolutionTitle)
    #print("---")
  print("================")    

#all the parameters within the XSLT - when was - decisions > elections/appointments who is heading this committee / what are the countries part of this committee
#resolutions xslt first > majority of items - then decisions
#what resolutions were adopted in May about water / oceans resolutions

# query_dict={}
# query_dict["sdg"]='sdg_2'
# query_dict["keyword"]=None
# query_dict["voted_for"]=None
# query_dict["voted_against"]='AFGHANISTAN' #'ISRAEL'
# query_dict["abstained"]=None

# output_res_dict={}
# for query_key,query_val in query_dict.items():
#   output_res_dict[query_key]=final_inv_index_dict[query_key].get(query_val,[])

# cur_vals=[]
# for query_key,query_val in query_dict.items():
#   if query_val==None: continue
#   query_res=output_res_dict.get(query_key,[])
#   #print(query_key,query_val,len(query_res),query_res[:10])
#   cur_vals.append(query_res)

# combined=AND(cur_vals)
# print("Number of results",len(combined))
# print("================")
# for res_path0 in combined:
#   #print(res_path0)
#   cur_data0=main_data_dict[res_path0]
#   resolutionTitle=cur_data0.get("resolutionTitle")
#   docnumber=cur_data0.get("docnumber")
#   print(docnumber)
#   print(resolutionTitle)
#   print("---")


sdg_1
Number of results 2
A/RES/77/48 Promotion of multilateralism in the area of disarmament and non-proliferation
A/RES/77/131 Implementation of the Declaration on the Granting of Independence to Colonial Countries and Peoples by the specialized agencies and the international institutions associated with the United Nations
sdg_2
Number of results 5
A/RES/77/53 Humanitarian consequences of nuclear weapons
A/RES/77/175 Role of the United Nations in promoting development in the context of globalization and interdependence
A/RES/77/123 Assistance to Palestine refugees
A/RES/77/13 Cooperation between the United Nations and the Collective Security Treaty Organization
A/RES/77/212 The right to development
sdg_3
Number of results 5
A/RES/77/175 Role of the United Nations in promoting development in the context of globalization and interdependence
A/RES/77/123 Assistance to Palestine refugees
A/RES/77/13 Cooperation between the United Nations and the Collective Security Treaty Organization
A/

In [ ]:
query_key0="sdg_2"
cur_results=inv_index.get(query_key0,[])
print("query_key0",query_key0)
print("number of results: %s"%len(output0))
for result0 in cur_results:
  cur_doc_data_dict=main_data_dict.get(result0,{})
  for key0,val0 in cur_doc_data_dict.items():
    if key0=="keywords":
      for v0 in val0: print(v0)
    else: print(key0,val0)
  print("-----")

query_key0 sdg_2
number of results: 24
docnumber A/RES/77/10
doctitle Resolution adopted by the General Assembly on 10 November 2022
resolutionTitle The situation in Afghanistan
AFGHAN REFUGEES
AFGHANISTAN
AFGHANISTAN SITUATION
COUNTER-TERRORISM
DRUG CONTROL
ECONOMIC DEVELOPMENT
GOVERNANCE
HUMAN RIGHTS
HUMANITARIAN ASSISTANCE
INTERNAL SECURITY
RECONCILIATION
REGIONAL COOPERATION
RULE OF LAW
Taliban (Afghanistan)
The situation in Afghanistan.
UN Assistance Mission in Afghanistan
civil_society
food_security
food_shortage
gender_equality
girls
health
hunger
justice
participation_of_women
partnership
peaceful_coexistence
position_of_women
public_health
public_institution
right_to_justice
sdg_16
sdg_17
sdg_2
sdg_3
sdg_5
social_justice
sustainable_agriculture
sustainable_development
women
women's_rights
-----
docnumber A/RES/77/100
doctitle Resolution adopted by the General Assembly on 7 December 2022
resolutionTitle United Nations Convention on the International Effects of Judicial Sales of

In [ ]:
for a in fwd_index:
  print(a)

('AFGHAN REFUGEES', 'output/res_json/A_RES_77_10-2225538E.json')
('AFGHANISTAN', 'output/res_json/A_RES_77_10-2225538E.json')
('AFGHANISTAN SITUATION', 'output/res_json/A_RES_77_10-2225538E.json')
('ARMED CONFLICTS', 'output/res_json/A_RES_77_104-2228250E.json')
('Articles on Diplomatic Protection (Draft)', 'output/res_json/A_RES_77_105-2228251E.json')
('Articles on Prevention of Transboundary Damage from Hazardous Activities (Draft)', 'output/res_json/A_RES_77_106-2228252E.json')
('COUNTER-TERRORISM', 'output/res_json/A_RES_77_10-2225538E.json')
('Consideration of prevention of transboundary harm from hazardous activities and allocation of loss in the case of such harm.', 'output/res_json/A_RES_77_106-2228252E.json')
('DEVELOPING COUNTRIES', 'output/res_json/A_RES_77_102-2228247E.json')
('DIPLOMATIC PROTECTION', 'output/res_json/A_RES_77_105-2228251E.json')
('DISASTER PREPAREDNESS', 'output/res_json/A_RES_77_1-2223391E.json')
('DISASTER PREVENTION', 'output/res_json/A_RES_77_1-2223391

In [ ]:
!pip install sqlitedict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sqlitedict: filename=sqlitedict-2.1.0-py3-none-any.whl size=16863 sha256=61bc2fa509f48c73f8487f1a7551be7c29fdeb51a283ed8bf266579d4e29d8ed
  Stored in directory: /root/.cache/pip/wheels/79/d6/e7/304e0e6cb2221022c26d8161f7c23cd4f259a9e41e8bbcfabd
Successfully built sqlitedict


In [ ]:
import lxml.etree as ET

xml_fpath='GAresolutions/77session/English/A_RES_77_149-2228534E.xml'
xslt_fpath="xslt/GA.v1.xslt"
dom = ET.parse(xml_fpath)
xslt = ET.parse(xslt_fpath)
transform = ET.XSLT(xslt)
newdom = transform(dom)
print(ET.tostring(newdom, pretty_print=True))

XSLTParseError: ignored

In [ ]:
items=list(dom_obj.tag_dict.items())
for a,b in items:
  if a=="": continue
  if a.startswith("text_node"): continue
  print(a,b.attrs)
  print([b.inner_html[:200]])
  # print(a,b.open_tag, b.attrs)
  
  print("----")
  

akomantoso_0 {'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', 'xsi:schemaLocation': 'http://docs.oasis-open.org/legaldocml/ns/akn/3.0 http://docs.oasis-open.org/legaldocml/akn-core/v1.0/os/part2-specs/schemas/akomantoso30.xsd', 'xmlns': 'http://docs.oasis-open.org/legaldocml/ns/akn/3.0', 'xmlns:html': 'http://www.w3.org/1999/xhtml', 'xmlns:akn4un': 'http://un.org/akn4un'}
['\n  <!--This is an unofficial copy for informational purposes only and prepared from the official electronic file-->\n  <statement name="resolution">\n    <meta>\n      <identification source="#DGACM">\n  ']
----
!--this_0 {}
['']
----
statement_0 {'name': 'resolution'}
['\n    <meta>\n      <identification source="#DGACM">\n        <FRBRWork>\n          <FRBRthis value="/akn/un/statement/deliberation/unga/2022-12-07/77-100/!main/"></FRBRthis>\n          <FRBRuri value="/ak']
----
meta_0 {}
['']
----
identification_0 {'source': '#DGACM'}
['\n        <FRBRWork>\n          <FRBRthis value="/akn/un/statement/

#Automatic pre-editing

In [ ]:
from code_utils.cat_utils import *
from code_utils.web_lib import *
from code_utils import web_lib

def get_docx_paras_edits(docx_fpath):
  docx_obj=docx(docx_fpath)
  data_list1=[]
  all_paras,paras_dict=docx_obj.extract_paras()
  docx_obj.close()
  for para_path0,para_content0 in all_paras:
    orig0,final0,edit0=get_edit_info(para_content0)
    if orig0==final0=="": continue
    data_list1.append((para_path0,orig0,final0,edit0))
  return data_list1


# 2023/Files/2306320E/2306320 EC22023R2Add23 MS.docx 
# 2023/Files/2306390E(asteric)/2306390E_Editing.doc
# 2023/Files/2306401E/2306401E_Editing.docx 
# 2023/Files/2306422E/2306422E_Editing.docx

fpath="2308011E_Editing.docx"
fpath="2023/Files/2306422E/2306422E_Editing.docx"
#fpath='2023/Files/2306320E/2306320 EC22023R2Add23 MS.docx'
docx_obj=docx(fpath)
data_list1=[]
all_paras,paras_dict=docx_obj.extract_paras()

for para_path0,para_content0 in all_paras:
  #print(para_content0)
  orig0,final0,edit0=get_edit_info(para_content0)
  if orig0==final0=="": continue
  print("para_path0",para_path0)
  print("orig0",orig0)
  print("final0",final0)
  print("edit0",edit0)
  print("-----")
  data_list1.append((orig0,final0,edit0))



table_content0=""
for i0,tag_item0 in enumerate(data_list1):
  original0,final0,edited0=tag_item0
  table_class="table-light"
  if i0%2!=0: table_class="table-dark text-dark"
  cur_tr0='<tr class="%s"><td>%s</td><td>%s</td><td>%s</td></tr>'%(table_class,original0,final0,edited0)
  table_content0+=cur_tr0

template_fpath="pre-editing_table_template.html"
template_fopen=open(template_fpath)
template_content=template_fopen.read()
template_fopen.close()
template_dom_obj=DOM(template_content)
repl_dict={"#data_display_table_body":table_content0}
out_html=template_dom_obj.replace(repl_dict)

out_fopen=open("pre-edit-initial2.html","w")
out_fopen.write(out_html)
out_fopen.close()


para_path0 word/document.xml|edb1a7d133
orig0 Letter dated 3 April 2023 from the Permanent Representative of the Russian Federation to the United Nations addressed to the Secretary-General
final0 Letter dated 3 April 2023 from the Permanent Representative of the Russian Federation to the United Nations addressed to the Secretary-General
edit0 Letter dated 3 April 2023 from the Permanent Representative of the Russian Federation to the United Nations addressed to the Secretary-General
-----
para_path0 word/document.xml|a4dcfee890
orig0 Note by the Secretary-General
final0 Note by the Secretary-General
edit0 Note by the Secretary-General
-----
para_path0 word/document.xml|7ed7569d29
orig0 Committee on Relations with the Host Country
final0 Committee on Relations with the Host Country
edit0 Committee on Relations with the Host Country
-----
para_path0 word/document.xml|e58b0af5e2
orig0 The Secretary-General has the honour to draw the attention of the members of the Committee on Relations w

In [ ]:
from code_utils.general import *
editing_extract_dir="output/edit_extracts"
if not os.path.exists(editing_extract_dir): os.makedirs(editing_extract_dir)
files=get_dir_files("2023","docx")
editing_files=[v for v in files if "Editing" in v]
for fpath0 in editing_files:
  print(fpath0)
  fname0=os.path.split(fpath0)[-1]
  extract_fname0=fname0.replace(".docx",".tsv")
  out_fpath=os.path.join(editing_extract_dir,extract_fname0)
  if os.path.exists(out_fpath): continue
  out_content=get_docx_paras_edits(fpath0)
  out_fopen=open(out_fpath,"w")
  for item0 in out_content: 
    item0=[v.replace("\t","  ") for v in item0]
    out_fopen.write("\t".join(item0)+"\n")
  out_fopen.close()
  #create_tsv(out_fpath,out_content)


# editing_files=[v for v in files if "Editing" in v]
# print(len(editing_files))

2023/Files/2300210E/2300210E_Editing.docx
2023/Files/2300271E/2300271E_Editing.docx
2023/Files/2300270E/2300270E_Editing.docx
2023/Files/2300286E/2300286E_Editing.docx
2023/Files/2300398E/2300398E_Editing.docx
2023/Files/2300500E/2300500E_Editing.docx
2023/Files/2300612E/2300612E_Editing.docx
2023/Files/2300744E/2300744E_Editing.docx
2023/Files/2300697R/2300697R_Editing.docx
2023/Files/2300773E/2300773E_Editing.docx
2023/Files/2300838E/2300838E_Editing.docx
2023/Files/2300917E/2300917E_NYVN_Editing.docx
2023/Files/2300854R/2300854R_Editing.docx
2023/Files/2300853E/2300853E_Editing.docx
2023/Files/2300856E/2300856E_Editing.docx
2023/Files/2300857E/2300857E_Editing.docx
2023/Files/2300906E/2300906E_Editing.docx
2023/Files/2300980E/2300980E_Editing.docx
2023/Files/2301023E/2301023E_Editing.docx
2023/Files/2301028E/2301028E_Editing.docx
2023/Files/2301030E/2301030E_Editing.docx
2023/Files/2301047E/2301047E_Editing.docx
2023/Files/2301308E/2301308E_Editing.docx
2023/Files/2301291E/2301291E_

In [ ]:
for f in files[:100]:
  print(f)
len(files)
#Vol disregard - cannot be edited for content

2023/Files/2300181E/2300181 A77684 IR.docx
2023/Files/2300204E/2300204 S202319 AR KB.docx
2023/Files/2300197E/2300197 S202341 AF.docx
2023/Files/2300193E/E-12-GA 77 Vol. I - Annex I edited HJ.docx
2023/Files/2300193E/E-04-GA 77 Vol. I - WR edited HJ.docx
2023/Files/2300193E/E-08-GA 77 Vol. I - 2C edited HJ.docx
2023/Files/2300193E/E-06-GA 77 Vol. I - 4C edited HJ.docx
2023/Files/2300193E/E-13-GA 77 Vol. I - Annex II edited HJ.docx
2023/Files/2300193E/E-09-GA 77 Vol. I - 3C edited HJ.docx
2023/Files/2300193E/E-07-GA 77 Vol. I - 6C edited HJ.docx
2023/Files/2300193E/E-05-GA 77 Vol. I - 1C edited HJ.docx
2023/Files/2300193E/E-10-GA 77 Vol. I - 5C edited HJ.docx
2023/Files/2300213E/2300213 AC.177INF3Rev.1 AR.docx
2023/Files/2300210E/2300210E_Editing.docx
2023/Files/2300220E/2300220 A77L44 FGE.docx
2023/Files/2300220E/2300220 A77L44 dept-ed corr 11Jan 9.35 FGE.docx
2023/Files/2300221E/2300221 A775 Vol II IR ed and dept corr 4 April.docx.docx
2023/Files/2300221E/2300221 A775 Vol II IR ed and

1695

In [ ]:
from code_utils.web_lib import *
xml_fpath="test_files/document.xml"
#full_path=os.path.join(cwd,xml_fpath)
fopen=open(xml_fpath)
xml_content=fopen.read()
fopen.close()
dom_obj=DOM(xml_content)
print(xml_content[:100])

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<w:document xmlns:wpc="http://schemas.micros


In [ ]:
from code_utils.general import *
tag_dict0=dom_obj.tag_dict
items=list(tag_dict0.items())
for a,b in items[:2000]:
  if a.startswith('w:document'): continue
  if a.startswith('w:body'): continue
  if not a.startswith("w:p_"): continue
  outer0=b.outer_html
  text0=remove_tags(outer0)
  #outer0=b.inner_html
  if len(outer0)==0: continue
  print(a, outer0) 
  print("-----")
  
  

w:p_0 <w:p w14:paraId="76DE9A46" w14:textId="77777777" w:rsidR="00596C1B" w:rsidRDefault="00596C1B" w:rsidP="00DB1BE0"><w:pPr><w:pStyle w:val="Header"/><w:spacing w:after="120"/><w:rPr><w:ins w:id="1" w:author="Katherine Bernabe" w:date="2023-05-04T12:15:00Z"/></w:rPr></w:pPr><w:bookmarkStart w:id="2" w:name="_Hlk115160567"/></w:p>
-----
w:p_2 <w:p w14:paraId="54D15974" w14:textId="77777777" w:rsidR="00596C1B" w:rsidRPr="00EA6FD5" w:rsidRDefault="00596C1B" w:rsidP="00DB1BE0"><w:pPr><w:pStyle w:val="HCh"/><w:spacing w:after="80"/><w:rPr><w:ins w:id="3" w:author="Katherine Bernabe" w:date="2023-05-04T12:15:00Z"/><w:b w:val="0"/><w:spacing w:val="2"/><w:w w:val="96"/></w:rPr></w:pPr><w:ins w:id="4" w:author="Katherine Bernabe" w:date="2023-05-04T12:15:00Z"><w:r w:rsidRPr="00EA6FD5"><w:rPr><w:b w:val="0"/><w:spacing w:val="2"/><w:w w:val="96"/></w:rPr><w:t>United Nations</w:t></w:r></w:ins></w:p>
-----
w:p_4 <w:p w14:paraId="7E6191E0" w14:textId="77777777" w:rsidR="00596C1B" w:rsidRPr="00E

In [ ]:
import re
from itertools import groupby
from code_utils.web_lib import *
content0='''<w:p w14:paraId="7F8B6662" w14:textId="0293594B" w:rsidR="004B6B95" w:rsidRPr="00EA6FD5" w:rsidRDefault="00881A4F"><w:pPr><w:pStyle w:val="ListParagraph"/><w:pBdr><w:top w:val="single" w:sz="4" w:space="1" w:color="auto"/><w:left w:val="single" w:sz="4" w:space="4" w:color="auto"/><w:bottom w:val="single" w:sz="4" w:space="1" w:color="auto"/><w:right w:val="single" w:sz="4" w:space="4" w:color="auto"/></w:pBdr><w:spacing w:before="240"/><w:ind w:left="360" w:firstLine="360"/><w:contextualSpacing w:val="0"/><w:jc w:val="both"/><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:pPrChange w:id="141" w:author="Sara Boyes" w:date="2023-05-08T17:21:00Z"><w:pPr><w:pStyle w:val="ListParagraph"/><w:numPr><w:numId w:val="31"/></w:numPr><w:spacing w:before="240"/><w:ind w:left="360" w:hanging="360"/><w:contextualSpacing w:val="0"/><w:jc w:val="both"/></w:pPr></w:pPrChange></w:pPr><w:r w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t>However,</w:t></w:r><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve"> the President and the two Vice-Presidents </w:t></w:r><w:r w:rsidR="004B1EC2" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve">of the Federation </w:t></w:r><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve">have not yet agreed on forming a new government. I encouraged all political parties to reach a compromise. I emphasized that a long-standing political deadlock in the Federation </w:t></w:r><w:del w:id="142" w:author="Sara Boyes" w:date="2023-05-08T17:38:00Z"><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5" w:rsidDel="00076A6D"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:delText xml:space="preserve">is </w:delText></w:r></w:del><w:ins w:id="143" w:author="Sara Boyes" w:date="2023-05-08T17:38:00Z"><w:r w:rsidR="00076A6D" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve">was </w:t></w:r></w:ins><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve">detrimental to Bosnia and Herzegovina </w:t></w:r><w:del w:id="144" w:author="Sara Boyes" w:date="2023-05-08T17:27:00Z"><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5" w:rsidDel="007F0D33"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:delText>(</w:delText></w:r></w:del><w:del w:id="145" w:author="Sara Boyes" w:date="2023-05-08T17:26:00Z"><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5" w:rsidDel="007F0D33"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:delText>BiH</w:delText></w:r></w:del><w:del w:id="146" w:author="Sara Boyes" w:date="2023-05-08T17:27:00Z"><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5" w:rsidDel="007F0D33"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:delText xml:space="preserve">) </w:delText></w:r></w:del><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t>and its</w:t></w:r><w:r w:rsidR="004B1EC2" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve"> path to</w:t></w:r><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve"> </w:t></w:r><w:del w:id="147" w:author="Sara Boyes" w:date="2023-05-08T17:36:00Z"><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5" w:rsidDel="0036450E"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:delText>EU</w:delText></w:r></w:del><w:del w:id="148" w:author="Sara Boyes" w:date="2023-05-08T18:33:00Z"><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5" w:rsidDel="00831E44"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:delText xml:space="preserve"> </w:delText></w:r></w:del><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t>integration</w:t></w:r><w:ins w:id="149" w:author="Sara Boyes" w:date="2023-05-08T18:33:00Z"><w:r w:rsidR="00831E44" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve"> in</w:t></w:r></w:ins><w:ins w:id="150" w:author="Sara Boyes" w:date="2023-05-10T17:04:00Z"><w:r w:rsidR="00D61122" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t>to</w:t></w:r></w:ins><w:ins w:id="151" w:author="Sara Boyes" w:date="2023-05-08T18:33:00Z"><w:r w:rsidR="00831E44" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t xml:space="preserve"> the European Union</w:t></w:r></w:ins><w:r w:rsidR="004B6B95" w:rsidRPr="00EA6FD5"><w:rPr><w:rFonts w:ascii="Times New Roman" w:hAnsi="Times New Roman" w:cs="Times New Roman"/><w:bCs/><w:sz w:val="24"/><w:szCs w:val="24"/><w:shd w:val="clear" w:color="auto" w:fill="FFFFFF"/></w:rPr><w:t>. The continuation of the current caretaker government would lack democratic legitimacy.</w:t></w:r></w:p>'''

def get_edit_info(para_content):
  tags=list(re.finditer('<[^<>]*?>|\<\!\-\-.+?\-\-\>', para_content))
  open_tags=[""]
  tag_counter_dict={}
  start_i=0
  last_open_tag_str=""
  is_inserted=False
  is_deleted=False
  original_text0,final_text0="",""
  edit_segments0=[]
  for ti_, t in enumerate(tags):
    tag_str,tag_start,tag_end=t.group(0), t.start(), t.end()
    tag_str_lower=tag_str.lower()
    tag_name=re.findall(r'</?(.+?)[\s>]',tag_str_lower)[0]
    tag_type=""
    if tag_str.startswith('</'): tag_type="closing"
    elif tag_str.startswith('<!'): tag_type="comment"
    elif tag_str_lower.endswith('/>') or tag_name in ["input","link","meta","img","br","hr"]: tag_type="s" #standalone
    else: tag_type="opening"
    if tag_name=="w:ins" and tag_type=="opening": is_inserted=True
    if tag_name=="w:ins" and tag_type=="closing": is_inserted=False
    if tag_name=="w:del" and tag_type=="opening": is_deleted=True
    if tag_name=="w:del" and tag_type=="closing": is_deleted=False

    inter_text=para_content[start_i:tag_start] #intervening text since last tag
    #print(tag_name, inter_text,"is_inserted",is_inserted,"is_deleted",is_deleted)
    if not is_inserted: original_text0+=inter_text
    if not is_deleted: final_text0+=inter_text
    seg_class="edited_same"
    if is_inserted: seg_class="edited_inserted"
    if is_deleted: seg_class="edited_deleted"
    if inter_text!="":edit_segments0.append((inter_text,seg_class))
    start_i=tag_end
  #edit_segments_grouped=edit_segments0[(key,"".join([v[0] for v in list(group)])) for key,group in groupby(edit_segments0,lambda x,x[0])]
  edit_segments_grouped0=[(key,"".join([v[0] for v in list(group)])) for key,group in groupby(edit_segments0,lambda x:x[1])]
  edited_text_html0=""
  for a,b in edit_segments_grouped0:
    if a=="edited_inserted": edited_text_html0+='<ins>'+b+'</ins>'
    elif a=="edited_deleted": edited_text_html0+='<del>'+b+'</del>'
    else: edited_text_html0+=b
  return original_text0,final_text0, edited_text_html0

def get_xml_paras(xml_content):
  dom_obj=DOM(xml_content)
  dom_items=list(dom_obj.tag_dict.items())
  dom_items=[b.outer_html for a,b in dom_items if a.startswith("w:p_")]
  return dom_items

test_xml_fpath="test_files/document0.xml"
fopen=open(test_xml_fpath)
xml_content0=fopen.read()
fopen.close()

data_list1=[]
#xml_paras0=get_xml_paras(xml_content0)
xml_paras0=get_xml_paras(xml_content0)
for para1 in xml_paras0:
  original_text1,final_text1, edit_text_html1=get_edit_info(para1)
  if original_text1==final_text1=="": continue
  print("original_text1:",original_text1)
  print("final_text1:",final_text1)
  print("edit_text_html1:",edit_text_html1)
  #for e_seg0 in edit_segments0: print(e_seg0)
  print("---")
  data_list1.append((original_text1,final_text1, edit_text_html1))


table_content0=""
for i0,tag_item0 in enumerate(data_list1):
  original0,final0,edited0=tag_item0
  table_class="table-light"
  if i0%2!=0: table_class="table-dark text-dark"

  cur_tr0='<tr class="%s"><td>%s</td><td>%s</td><td>%s</td></tr>'%(table_class,original0,final0,edited0)
  table_content0+=cur_tr0



template_fpath="pre-editing_table_template.html"
template_fopen=open(template_fpath)
template_content=template_fopen.read()
template_fopen.close()
template_dom_obj=DOM(template_content)
repl_dict={"#data_display_table_body":table_content0}
out_html=template_dom_obj.replace(repl_dict)

out_fopen=open("pre-edit-initial.html","w")
out_fopen.write(out_html)
out_fopen.close()


# tags=list(re.finditer('<[^<>]*?>|\<\!\-\-.+?\-\-\>', content0))
# open_tags=[""]
# tag_counter_dict={}
# start_i=0
# last_open_tag_str=""
# is_inserted=False
# is_deleted=False
# original_text,final_text="",""
# for ti_, t in enumerate(tags):
#   tag_str,tag_start,tag_end=t.group(0), t.start(), t.end()
#   tag_str_lower=tag_str.lower()
#   tag_name=re.findall(r'</?(.+?)[\s>]',tag_str_lower)[0]
#   tag_type=""
#   if tag_str.startswith('</'): tag_type="closing"
#   elif tag_str.startswith('<!'): tag_type="comment"
#   elif tag_str_lower.endswith('/>') or tag_name in ["input","link","meta","img","br","hr"]: tag_type="s" #standalone
#   else: tag_type="opening"
#   if tag_name=="w:ins" and tag_type=="opening": is_inserted=True
#   if tag_name=="w:ins" and tag_type=="closing": is_inserted=False
#   if tag_name=="w:del" and tag_type=="opening": is_deleted=True
#   if tag_name=="w:del" and tag_type=="closing": is_deleted=False

#   inter_text=content0[start_i:tag_start] #intervening text since last tag
#   #print(tag_name, inter_text,"is_inserted",is_inserted,"is_deleted",is_deleted)
#   if not is_inserted: original_text+=inter_text
#   if not is_deleted: final_text+=inter_text
#   start_i=tag_end
# original_text,final_text, edit_segments=get_edit_info(content0)
# print("original_text:")
# print(original_text)
# print("final_text:")
# print(final_text)
# print("edit_segments:")
# for a in edit_segments: print(a)
# #print(edit_segments)

original_text1: United Nations
final_text1: United Nations
edit_text_html1: United Nations
---
original_text1: S/2021/200
final_text1: S/2021/200
edit_text_html1: S/2021/200
---
original_text1: Security Council
final_text1: Security Council
edit_text_html1: Security Council
---
original_text1: Distr.: General
final_text1: Distr.: General
edit_text_html1: Distr.: General
---
original_text1: 1 March 2021
final_text1: 1 March 2021
edit_text_html1: 1 March 2021
---
original_text1: Original: English
final_text1: Original: English
edit_text_html1: Original: English
---
original_text1: Letter dated 26 February 2021 from the Secretary-General addressed to the President of the Security Council
final_text1: Letter dated 26 February 2021 from the Secretary-General addressed to the President of the Security Council
edit_text_html1: Letter dated 26 February 2021 from the Secretary-General addressed to the President of the Security Council
---
original_text1: I have the honour to transmit herewith t

In [ ]:
!pwd
!ls test_files

/content/drive/MyDrive/AI_UN
A_RES_77_10-2225538E.docx  document0.xml  edited-text-sample.docx
A_RES_77_10-2225538E.xml   document.xml


In [ ]:
from code_utils.web_lib import *
test_xml_fpath="test_files/document.xml"
fopen=open(test_xml_fpath)
xml_content=fopen.read()
fopen.close()

dom_obj=DOM(xml_content)
dom_items=list(dom_obj.tag_dict.items())
dom_items=[(a,b) for a,b in dom_items if a.startswith("w:p_")]
for a,b in dom_items:
  #print(a,b.outer_html)
  orig0,edited0,segs0=get_edit_info(b.outer_html)
  if orig0=="" and edited0=="": continue
  print(orig0)
  print(edited0)
  print(segs0)
  print("----")


United Nations
[('edited_inserted', 'United Nations')]
----

S/2023/318
[('edited_inserted', 'S/2023/318')]
----

Security Council
[('edited_inserted', 'Security Council')]
----

Distr.: General
[('edited_inserted', 'Distr.: General')]
----

4 May 2023
[('edited_inserted', '4 May 2023')]
----

Original: French
[('edited_inserted', 'Original: French')]
----

Letter dated 3 May 2023 from the Secretary-General addressed to the President of the Security Council
[('edited_inserted', 'Letter dated 3 May 2023 from the Secretary-General addressed to the President of the Security Council')]
----
Le 3 mai 2023

[('edited_deleted', 'Le 3 mai 2023')]
----
Madame la Présidente, 

[('edited_deleted', 'Madame la Présidente,\xa0')]
----
 
 
[('edited_same', '\xa0')]
----
Conformément à la résolution 1031 (1995) du Conseil de sécurité, 
Conformément à la résolution 1031 (1995) du Conseil de sécurité, 
[('edited_same', 'Conformément à la résolution 1031 (1995) du Conseil de sécurité, ')]
----
j’ai l’ho

In [ ]:
!unzip data/2021.zip
!unzip data/2022.zip
!unzip data/2023.zip

Streaming output truncated to the last 5000 lines.
  inflating: 2022/Files/2226378E/2226378 S2022871 SB.docx  
   creating: 2022/Files/2226381E/
  inflating: 2022/Files/2226381E/2226381 A777Add21 AC Ed corr1 29 Nov.pdf  
  inflating: 2022/Files/2226381E/2226381 A777Add21 AC.docx  
   creating: 2022/Files/2226383E/
  inflating: 2022/Files/2226383E/2226383 A777Add22 AC - dept. corr2. AC 1 Dec.pdf  
  inflating: 2022/Files/2226383E/2226383 A777Add22 AC.docx  
  inflating: 2022/Files/2226383E/2226383 A777Add22 with dept corr IR ed corr 1 Dec.pdf  
  inflating: 2022/Files/2226383E/2226383 A777Add22 with dept corr IR.docx  
   creating: 2022/Files/2226386E/
  inflating: 2022/Files/2226386E/2226386 A777Add23 28Nov 14.45 dept corr FGE.pdf  
  inflating: 2022/Files/2226386E/2226386 A777Add23 29Nov 16.30 ed corr FGE.pdf  
  inflating: 2022/Files/2226386E/2226386 A777Add23 FGE.docx  
   creating: 2022/Files/2226387E/
  inflating: 2022/Files/2226387E/2226387 AC277L76 Dept Corr 22 November (13.35) 

#Processing docx

In [ ]:
import shutil, zipfile
import hashlib
from code_utils.web_lib import *

def simple_hash(input_str,size=6):
  input_str=input_str.encode('utf-8')
  return hashlib.md5(input_str).hexdigest()[:size]


class docx:
	def __init__(self,docx_fpath,keep_copy=True): #openning the docx file, by unzipping it
		self.TEMP_DOCX = docx_fpath
		self.closed=False
		self.COPY_DOCX = docx_fpath+"2"
		shutil.copy(self.TEMP_DOCX, self.COPY_DOCX) #keep a temp copy of out file, just in case
		self.file_extension="."+docx_fpath.split(".")[-1]
		self.TEMP_ZIP = docx_fpath.replace(self.file_extension,".zip")
		self.TEMP_FOLDER = docx_fpath.replace(self.file_extension,"")
		if os.path.exists(self.TEMP_ZIP):
			os.remove(self.TEMP_ZIP)
		if os.path.exists(self.TEMP_FOLDER):
			shutil.rmtree(self.TEMP_FOLDER)
		os.rename(self.TEMP_DOCX, self.TEMP_ZIP) #rename the original docx to zip extension
		# unzip file zip to specific folder
		z_open=zipfile.ZipFile(self.TEMP_ZIP, 'r')
		z_open.extractall(self.TEMP_FOLDER)
		z_open.close()
		os.rename(self.COPY_DOCX, self.TEMP_DOCX) #keep the original file
	def save_as(self,out_fpath):
		if os.path.exists(out_fpath): #remove any of these if already exists
			os.remove(out_fpath)
		#self.OUT_ZIP = out_fpath.replace(".docx",".zip")
		#shutil.make_archive(self.OUT_ZIP, 'zip', self.TEMP_FOLDER)
		shutil.make_archive(self.TEMP_ZIP.replace(".zip", ""), 'zip', self.TEMP_FOLDER)
		os.rename(self.TEMP_ZIP, out_fpath)
	def extract_paras(self,cat_file_path=""): 
		self.paras=[]
		self.para_path_dict={}
		if self.TEMP_DOCX.lower().endswith(".docx"): main_dir="word"
		if self.TEMP_DOCX.lower().endswith(".pptx"): main_dir="ppt/slides"
		if self.TEMP_DOCX.lower().endswith(".xlsx"): main_dir="xl"

		extracted_dir=os.path.join(self.TEMP_FOLDER, main_dir)
		for xml_fname in os.listdir(extracted_dir):
			#print(xml_fname)
			skip_file=True
			if xml_fname in ["document.xml", "footnotes.xml","endnotes.xml","sharedStrings.xml"]: skip_file=False
			if xml_fname.startswith("slide"): skip_file=False
			if xml_fname.startswith("sheet"): skip_file=False
			if xml_fname.startswith("header"): skip_file=False
			if xml_fname.startswith("footer"): skip_file=False
			if skip_file: continue
			# if xml_fname in ["document.xml", "footnotes.xml","endnotes.xml"] or xml_fname.startswith("header") or xml_fname.startswith("footer"): pass
			# else: continue
			#if not xml_fname=="document.xml" and not xml_fname.startswith("header") and not xml_fname.startswith("footer"): continue
			cur_xml_path=os.path.join(extracted_dir,xml_fname)
			with open(cur_xml_path) as fopen:
				xml_content=fopen.read()
			xml_dom_obj=DOM(xml_content)
			cur_tag_name="w:p" #xml tag for docx files
			if self.TEMP_DOCX.lower().endswith(".pptx"): cur_tag_name="a:p"
			
			cur_paras=xml_dom_obj.get_el_by_tag_name(cur_tag_name)
			tmp_xml_path=os.path.join(main_dir,xml_fname)
			for para0 in cur_paras:
				para_hash=simple_hash(para0)
				cur_para_key="%s|%s"%(tmp_xml_path,para_hash)
				self.paras.append((cur_para_key,para0))
				self.para_path_dict[cur_para_key]=para0
			
			# cur_wps=get_xml_elements(xml_content,el_name=cur_tag_name)
			# for i0,wp_xml in enumerate(cur_wps):
			# 	#wp_text=get_wr_text(wp_xml)
			# 	wp_text=get_el_text(wp_xml)        
			# 	para_obj=para()
			# 	para_obj.path=cur_xml_path
			# 	para_obj.xml=wp_xml
			# 	para_obj.text=wp_text
			# 	para_obj.i=i0
			# 	para_obj.tag_name=cur_tag_name        
			# 	found_ids=re.findall('paraId="(.+?)"',wp_xml) 
			# 	if found_ids: para_obj.id=  found_ids[0]
			# 	self.paras.append(para_obj)
			# if cat_file_path!="": #save paragraphs with their info to cat file
			# 	cat_fopen=open(cat_file_path,"w")
			# 	for p_obj in self.paras:
			# 		cur_text=p_obj.text.replace("\t"," <tab> ").replace("\n"," <br> ")
			# 		json_obj={}
			# 		json_obj["path"]=p_obj.path
			# 		json_obj["i"]=p_obj.i
			# 		json_obj["id"]=p_obj.id
			# 		json_obj["tag_name"]=p_obj.tag_name          
			# 		json_obj["text"]=cur_text
			# 		json_obj_str=json.dumps(json_obj) 
			# 		line=json_obj_str+"\n"
			# 		#line="%s\t%s\t%s\t%s\n"%(p_obj.path,p_obj.i,p_obj.id,cur_text)
			# 		cat_fopen.write(line)
			# 	cat_fopen.close()

		return self.paras,self.para_path_dict
  

	def update_tbl_rtl(self):
		extracted_dir=os.path.join(self.TEMP_FOLDER, "word")
		cur_xml_path=os.path.join(extracted_dir,"document.xml")
		fopen_read=open(cur_xml_path)
		xml_content=fopen_read.read()
		fopen_read.close()
		xml_content=xml_content.replace("<w:tblPr>","<w:tblPr><w:bidiVisual/>")
		xml_content=xml_content.replace("<w:lang ","<w:rtl/><w:lang ")

		
		fopen_write=open(cur_xml_path, "wb")
		fopen_write.write(xml_content)
		fopen_write.close()


	def close(self):
		self.closed=True
		os.remove(self.TEMP_ZIP)
		shutil.make_archive(self.TEMP_ZIP.replace(".zip", ""), 'zip', self.TEMP_FOLDER)
		os.rename(self.TEMP_ZIP, self.TEMP_DOCX)
		shutil.rmtree(self.TEMP_FOLDER)

fpath="2308011E_Editing.docx"
docx_obj=docx(fpath)

In [ ]:
fpath="2308011E_Editing.docx"
docx_obj=docx(fpath)
all_paras,paras_dict=docx_obj.extract_paras()

In [ ]:
from code_utils.cat_utils import *

fpath="2308011E_Editing.docx"
docx_obj=docx(fpath)
all_paras,paras_dict=docx_obj.extract_paras()
for para_path0,para_content0 in all_paras[:100]:
  #print(para_content0)
  orig0,final0,edit0=get_edit_info(para_content0)
  if orig0==final0=="": continue
  print("para_path0",para_path0)
  print("orig0",orig0)
  print("final0",final0)
  print("edit0",edit0)
  print("-----")

para_path0 word/document.xml|44bebae4d9
orig0 Enclosure
final0 Enclosure
edit0 Enclosure
-----
para_path0 word/document.xml|f1e240c7ae
orig0 [Original: Arabic, Chinese, English, French, Russian and Spanish]
final0 [Original: Arabic, Chinese, English, French, Russian and Spanish]
edit0 [Original: Arabic, Chinese, English, French, Russian and Spanish]
-----
para_path0 word/document.xml|191c642649
orig0 OPCW
final0 
edit0 <del>OPCW</del>
-----
para_path0 word/document.xml|fc257be6aa
orig0 Executive Council
final0 
edit0 <del>Executive Council</del>
-----
para_path0 word/document.xml|553bd62240
orig0 103rd Session
final0 
edit0 <del>103rd Session</del>
-----
para_path0 word/document.xml|9fd76e66a9
orig0 11 – 14 July 2023
final0 
edit0 <del>11 – 14 July 2023</del>
-----
para_path0 word/document.xml|04d3deed31
orig0 EC-103/DG.3
final0 
edit0 <del>EC-103/DG.3</del>
-----
para_path0 word/document.xml|d5ea6840b9
orig0 25 April 2023
final0 
edit0 <del>25 April 2023</del>
-----
para_path0 word/do

##MT experiments

In [ ]:
#https://github.com/jadore801120/attention-is-all-you-need-pytorch
!rm -rf attention_mt
#!git clone https://github.com/jadore801120/attention-is-all-you-need-pytorch attention_mt
!git clone https://github.com/hmghaly/attention-is-all-you-need-pytorch.git attention_mt


Cloning into 'attention_mt'...
remote: Enumerating objects: 753, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 753 (delta 2), reused 0 (delta 0), pack-reused 747
Receiving objects: 100% (753/753), 170.15 KiB | 1.62 MiB/s, done.
Resolving deltas: 100% (493/493), done.


In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

2023-06-07 19:36:15.512036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 19:36:16.612112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 72.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2023-06-07 19:36:28.984037: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the

In [ ]:
!pip install dill
!!pip install torchtext==0.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.9 MB/s eta 0:00:00


['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting torchtext==0.4',
 '  Downloading torchtext-0.4.0-py3-none-any.whl (53 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/53.1 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━\x1b \x1b51.2/53.1 kB\x1b \x1b4.4 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b53.1/53.1 kB\x1b \x1b1.1 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hRequirement already satisfied: tqdm in /usr/local/lib/python3.10/dist-packages (from torchtext==0.4) (4.65.0)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.10/dist-packages (from torchtext==0.4) (2.27.1)',
 'Requirement already satisfied: torch in /usr/local/lib/python3.10/dist-packages (from torchtext==0.4) (2.0.1+cu118)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.10/

In [ ]:
os.chdir("attention_mt")
!python preprocess.py -lang_src de -lang_trg en -share_vocab -save_data m30k_deen_shr.pkl
os.chdir(cwd)

2023-06-07 19:36:53.277800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 19:36:54.385958: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Namespace(lang_src='de', lang_trg='en', save_data='m30k_deen_shr.pkl', data_src=None, data_trg=None, max_len=100, min_word_count=3, keep_case=False, share_vocab=True)
Traceback (most recent call last):
  File "/content/drive/MyDrive/AI_UN/attention_mt/preprocess.py", line 336, in <module>
    main_wo_bpe()
  File "/content/drive/MyDrive/AI_UN/attention_mt/preprocess.py", line 271, in main_wo_bpe
    src_lang_model = spacy.load(spacy_src_lang_model_path)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__init__.py", line 54, in load
    return util.load_

In [ ]:
import spacy
spacy.load('de_core_news_sm')

In [ ]:
spacy.load('en_core_web_sm')

In [ ]:
!!pip install torchtext==0.4

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Requirement already satisfied: torchtext==0.4 in /usr/local/lib/python3.10/dist-packages (0.4.0)',
 'Requirement already satisfied: tqdm in /usr/local/lib/python3.10/dist-packages (from torchtext==0.4) (4.65.0)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.10/dist-packages (from torchtext==0.4) (2.27.1)',
 'Requirement already satisfied: torch in /usr/local/lib/python3.10/dist-packages (from torchtext==0.4) (2.0.1+cu118)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.10/dist-packages (from torchtext==0.4) (1.22.4)',
 'Requirement already satisfied: six in /usr/local/lib/python3.10/dist-packages (from torchtext==0.4) (1.16.0)',
 'Requirement already satisfied: urllib3<1.27,>=1.21.1 in /usr/local/lib/python3.10/dist-packages (from requests->torchtext==0.4) (1.26.15)',
 'Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/pyth

In [ ]:
!pip install -U torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torchtext.data

In [ ]:
import torchtext.datasets as datasets

In [ ]:
datasets.

In [ ]:
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

ModuleNotFoundError: ignored

#OLD

In [ ]:
!pwd
!ls

/content/drive/MyDrive/AI_UN
 2015				    GAresolutions
 2021				    machine_readable_table_template.html
 2022				    mrdoc-initial.html
 2023				    ods-subject-list.xlsx
 2308011E_Editing.docx2		    OLD
 2308011E_Editing.zip		    Planning.gsheet
'AI Contact List - emails.gsheet'   pre-editing_table_template.html
'AI Strategy.gdoc'		    pre-edit-initial.html
 cat-mt.ipynb			   'Tentative Use Case Descriptions.gdoc'
 code_utils			    test_files
 data				   'Use Case Description - Full.gdoc'


In [ ]:
!cp 2023/Files/2308011E/2308011E_Editing.docx 2308011E_Editing.docx

In [ ]:
!ls /content/drive/MyDrive/AI_UN

 2015				    GAresolutions
 2021				    machine_readable_table_template.html
 2022				    mrdoc-initial.html
 2023				    ods-subject-list.xlsx
 2308011E_Editing.docx2		    OLD
 2308011E_Editing.zip		    Planning.gsheet
'AI Contact List - emails.gsheet'   pre-editing_table_template.html
'AI Strategy.gdoc'		    pre-edit-initial.html
 cat-mt.ipynb			   'Tentative Use Case Descriptions.gdoc'
 code_utils			    test_files
 data				   'Use Case Description - Full.gdoc'


In [ ]:
import shutil
root_dir=os.path.join(cwd,"Work-NLP")

fname="charts.pptx"
fname_copy="copy_"+fname
zip_fname=fname_copy.replace(".pptx",".zip")
unzip_dir=os.path.join(root_dir,fname.split(".")[0])
fpath=os.path.join(root_dir,fname)
fpath_copy=os.path.join(root_dir,fname_copy)
fpath_zip=os.path.join(root_dir,zip_fname)
shutil.copy(fpath,fpath_copy)
os.rename(fpath_copy,fpath_zip)

import zipfile
with zipfile.ZipFile(fpath_zip, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)


In [ ]:
import re
terms_fpath=os.path.join(root_dir,"org-charts-terms.tsv")
terms_fopen=open(terms_fpath)
repl_dict={}
for i,f in enumerate(terms_fopen):
  if i==0: continue
  f_split=f.strip("\n\r\t").split("\t")
  if len(f_split)!=2: continue
  en,ar=f_split
  if en==ar: continue
  repl_dict[en]=ar
  #print(f_split)
terms_fopen.close()


slides_dir=os.path.join(unzip_dir,"ppt","slides")
for slide_fname in os.listdir(slides_dir):
  if not slide_fname.endswith(".xml"): continue
  slide_fpath=os.path.join(slides_dir,slide_fname)
  print(slide_fpath)
  slide_fopen=open(slide_fpath)
  slide_content=slide_fopen.read()
  slide_fopen.close()
  segs=re.findall(r"<a:t\b.*?>.*?</a:t>",slide_content)
  for original_seg in segs:
    
    cur_seg=str(original_seg)
    print(original_seg)
    if re.findall(">\(\d+ posts?\)",cur_seg): 
      num=re.findall("\d+",cur_seg)[0]
      if int(num)==1:
        cur_seg="<a:t>"+num+ " وظيفة "+"</a:t>"
      elif int(num)>10:
        cur_seg="<a:t>"+num+ " وظيفة "+"</a:t>"
      else:
        cur_seg="<a:t>"+num+ " وظائف "+"</a:t>"
    
    else:
      cur_seg=cur_seg.replace(",","،")
      for r_en,r_ar in repl_dict.items():
        cur_seg=cur_seg.replace(r_en," "+ r_ar+" ")
      cur_seg=re.sub("\s+"," ",cur_seg)
    print(cur_seg)
    print("------")
    slide_content=slide_content.replace(original_seg,cur_seg)

  slide_fopen=open(slide_fpath,"w")
  slide_fopen.write(slide_content)
  #slide_content=slide_fopen.read()
  slide_fopen.close()

  #w_p_exp=r"<w:p\b.*?>.*?</w:p>"
  #<a:t>
import shutil
output_fname0="translated_charts"
output_fpath0=os.path.join(root_dir,output_fname0)
output_fpath_zip=os.path.join(output_fpath0+".zip")
output_fpath_pptx=os.path.join(output_fpath0+".pptx")
shutil.make_archive(output_fpath0, 'zip', unzip_dir)
os.rename(output_fpath_zip,output_fpath_pptx)

/content/drive/MyDrive/Work-NLP/charts/ppt/slides/slide1.xml
<a:t>Office of the Chief of Staff</a:t>
<a:t> مكتب رئيس الأركان </a:t>
------
<a:t>(13 posts)</a:t>
<a:t>13 وظيفة </a:t>
------
<a:t>1 D-2, 1 P-5, 1 P-4, 3 P-3, 2 P-2,</a:t>
<a:t>1 مد-2 ، 1 ف-5 ، 1 ف-4 ، 3 ف-3 ، 2 ف-2 ،</a:t>
------
<a:t>1 FS, 1 NPO, 3 NGS</a:t>
<a:t>1 خ م ، 1 م ف و ، 3 خ ع و </a:t>
------
<a:t>Joint Mission Analysis Centre</a:t>
<a:t> خلية التحليل المشتركة للبعثة </a:t>
------
<a:t>(7 posts)</a:t>
<a:t>7 وظائف </a:t>
------
<a:t>1 P-5, 2 P-4, 2 P-3, 2 NPO</a:t>
<a:t>1 ف-5 ، 2 ف-4 ، 2 ف-3 ، 2 م ف و </a:t>
------
<a:t>Conduct and Discipline Team</a:t>
<a:t> فريق السلوك والانضباط </a:t>
------
<a:t>(11 posts)</a:t>
<a:t>11 وظيفة </a:t>
------
<a:t>1 P-5, 1 P-4, 4 P-3, 1 FS, </a:t>
<a:t>1 ف-5 ، 1 ف-4 ، 4 ف-3 ، 1 خ م ، </a:t>
------
<a:t>3 NPO, 1 NGS</a:t>
<a:t>3 م ف و ، 1 خ ع و </a:t>
------
<a:t>Communications and Public</a:t>
<a:t>Communications and Public</a:t>
------
<a:t>Information Section </a:t>
<a:t>Info

In [ ]:
import hashlib
sha = hashlib.sha256()
sha.update('somestring'.encode())
sha.hexdigest()

def simple_hash(input_str,size=6):
  input_str=input_str.encode('utf-8')
  return hashlib.md5(input_str).hexdigest()[:size]

simple_hash("/content/drive/MyDrive/Work-NLP/charts/ppt/slides/slide1.xml",10)

'2c847cf6d9'